# Instrucciones.

Debería poder realizar todo el laboratorio en 3 horas.

**Recomendamos fuertemente** que siga trabajando en el laboratorio durante las horas asignadas, pues así puede consultar en vivo dudas que tenga sobre las partes finales del mismo. 

* Complete la celda "A. Datos grupo".
* Descargue los archivos de instancias de ucursos.
* Lea y ejecute la sección "B. Preparación" 
* Desarrolle la sección "C. Conjunto Independiente máximo de rectángulos (Modelo)" Incluye 1 ejercicio. No olvide responder la pregunta al final de la parte 1.c
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 3 presencial 
* Plazo de entrega:  Miércoles 17 de junio a las 18:00

Los ejercicios 2, 3 y 4  contenidos en las secciones D y E de este laboratorio se entregan este Viernes 19 de junio a las 22:00


* Complete todos los ejercicios (incluyendo las preguntas abiertas al final de algunos de ellos)
* Envie el archivo .ipynb por el modulo de tareas de ucursos en la TAREA: Laboratorio 3 completo
* Plazo de entrega: Viernes 19 de junio a las 22:00 
  (puede solicitar por correo electrónico plazo adicional si lo necesita, pero su solicitud debe hacerse antes del viernes  a las 22:00. No es automático)


# A. Datos grupo

Escriba en esta celda el nombre de cada integrante de su grupo.

- Integrante 1: XXX

- Integrante 2: YYY

 # B. Preparación.

### B1. Paquetes y Ambiente 

Usaremos los paquetes `JuMP, Gurobi, Plots, CSV, DataFrames`
   Asegúrese de tener la última versión de `JuMP` corriendo (0.21.2 en mayo 2020)
   Si le falta algún paquete instálelo (`using Pkg`; `Pkg.add("nombre_paquete")`) o actualícelo (`Pkg.update("nombre_paquete")`)
 
Al igual que en el laboratorio anterior, cree una constante `GUROBI_ENV` que guarde el ambiente gurobi que usaremos
(de modo de evitar múltiples instancias). 
 

In [ ]:
# using Pkg
# Pkg.add(" ")
# Pkg.update(" ")

using JuMP, Gurobi, Plots, CSV, DataFrames
const GUROBI_ENV = Gurobi.Env()

## B2. Rectángulos en el plano: definiciones y funciones auxiliares

Hoy trabajaremos con problemas relacionados con rectángulos en el plano.
   
Para $(a,b,c,d)\in \mathbb{Z}$, denotamos por $\text{rect}(a, b, c, d)=[a, c] \times [b, d] \subseteq \mathbb{Z}^2$  al rectángulo cerrado con lados alineados con los ejes, de esquina inferior izquierda $(a, b)$ y esquina superior derecha $(c, d)$. 
  
Describiremos familias de $n$ rectángulos como matrices $R\in \mathbb{Z}^{n\times 4}$ cuya $i$-ésima fila es $[a_i, b_i, c_i, d_i]$. El $i$-ésimo rectángulo de la familia es $\text{rect}(a_i,b_i,c_i,d_i)$
 
Revisar si dos rectángulos se intersectan es simple:
$$\text{rect}(a,b,c,d) \cap \text{rect}(a',b',c',d')\neq \emptyset \iff (\max(a,a')\leq \min(c,c')) \wedge (\max(b,b')\leq \min(c,c')$$  
 
Decimos que un conjunto $R$ de rectángulos es **independiente** si son disjuntos de a pares. 

Llamamos **área discreta** de un rectángulo $r$ al número de puntos enteros que este rectángulo contiene.

A continuación incluimos 4 funciones auxiliares para usar en este laboratoio

1. ``LeeArchivo(archivocsv)``
   Lee un archivo de nombre "archivocsv" que contiene una familia de rectángulo, devolviéndolo en formato matriz.
   
2. ``Intersecta(rect1,rect2)`` 
   Determina rápidamente si dos rectángulos se intersectan (devuelve true/false)
    
3. ``AreaDiscreta(rect)``
   Determina el área discreta del rectángulo
   
4. ``Dibuja(R)``
   Dibujar familias de rectángulos en el plano (les aumenta el grosor para permitir rectangulos de ancho o alto 0)
  

In [ ]:
function LeeArchivo(archivocsv)
    return Matrix(CSV.read(archivocsv,  header=0))
end

function Intersecta(rect1,rect2)
    return (max(rect1[1],rect2[1])<=min(rect1[3],rect2[3])) & (max(rect1[2],rect2[2])<=min(rect1[4],rect2[4]))
end

function AreaDiscreta(rect)
    return (rect[3]-rect[1]+1)*(rect[4]-rect[2]+1)
end

function Dibuja(R)
    N=maximum(R)
    plot()
    for i in 1:size(R,1)
       plot!(
            Shape([R[i,1]-0.1, R[i,3]+0.1, R[i,3]+0.1, R[i,1]-0.1], [R[i,2]-0.1, R[i,2]-0.1, R[i,4]+0.1, R[i,4]+0.1]),
            xlims=(0.85,N+0.15),
            ylims=(0.85,N+0.15),
            opacity=.5, legend=false, aspect_ratio=1)
    end
    display(plot!())
end

Como ejemplo, las siguientes lineas crean una matriz F de rectángulos a partir del archivo "rectangulos00.csv" y luego la dibujan

In [ ]:
F=LeeArchivo("rectangulos00.csv")

In [ ]:
Dibuja(F)

Por ejemplo, en el caso de la familia $F$ anterior, podemos calcular las áreas discretas de los 6 rectángulos como sigue:

In [ ]:
for i in 1:size(F,1)
    println("Rectangulo ", i, " cubre ", AreaDiscreta(F[i,:]), " puntos enteros")
end

Y podemos calcular la intersección par a par de los rectángulos de $F$ como sigue

In [ ]:
[Intersecta(F[i,:], F[j,:]) for i=1:size(F,1), j=1:size(F,1)]

Notamos que los rectangulos 1, 3, 4 y 5 forman un conjunto independiente. Podemos dibujar ese subconjunto escribiendo:

In [ ]:
Dibuja(F[[1, 3, 4, 5],:])

# C. Conjunto Independiente máximo de rectángulos (Modelo)

 El problema del **conjunto independiente máximo de rectángulos (MISR)** se define como sigue:
   
  **Entrada:**
  
  Una familia $R$ de $n$ rectángulos codificada como una matriz (del mismo nombre) $R\in \mathbb{Z}^{n\times 4}$ cuya $i$-ésima fila es $[a_i, b_i, c_i, d_i]$. 
  El $i$-ésimo rectángulo de la familia es $r_i=\text{rect}(a_i,b_i,c_i,d_i)$
  
  **Salida:**
  
  Un conjunto $X\subseteq R$ de rectángulos independientes de cardinal máximo.
  
  
  
  La formulación natural del problema como PLE es la siguiente (aquí $n$ es el número de rectángulos).
  
  $$(PE1)\quad \begin{align*}
  \max &\sum_{i=1} x_i\\
  x_i +x_j &\leq 1, \quad \forall 1\leq i<j\leq n: r_i\cap r_j \neq \emptyset\\
  x_i &\in \{0,1\}, \quad \forall i\in [n]
  \end{align*}
  $$
  
  también usaremos la siguiente relajación (PL1)
  $$(PL1)\quad \begin{align*}
  \max &\sum_{i=1} x_i\\
  x_i +x_j &\leq 1, \quad \forall 1\leq i<j\leq n: r_i\cap r_j \neq \emptyset\\
  0\leq x_i &\leq 1, \quad \forall i\in [n]
  \end{align*}
  $$
  
  
  

**IMPORTANTE:** 

En este laboratorio trabajaremos en resolver (PE1) para pequeñas instancias. Esta vez interactuaremos con el solver de distintas maneras: asignando prioridades, o cambiando características. ``JuMP`` no permite trabajar directamente con todas las funcionalidades de Gurobi, y es necesario usar paquetes de más bajo nivel como ``MathOptInterface`` (abreviado ``MOI``)

Para hacer todo más simple, usaremos la siguiente función que recibe un modelo y una matriz de dos columnas donde cada fila es de la forma ``["opcion", valor-opcion]`` que queramos pasarle a Gurobi. La lista de parámetros que Gurobi acepta está en  https://www.gurobi.com/documentation/9.0/refman/parameters.html

Por ejemplo, si m contiene un modelo y deseamos fijar el parámetro "TimeLimit" del solver a 30 segundos, y el parámetro "Presolve" a 0 podemos escribir
``Modifica(m, ["TimeLimit" 30; "Presolve" 0])``


In [ ]:
function Modifica(mimodelo, X)
    k=size(X,1)
    for j in 1:k
        MOI.set(mimodelo, MOI.RawParameter(X[j,1]), X[j,2])
        println("",X[j,1],"=>",X[j,2])
    end
end

## Ejercicio 1:

*  Los archivos "rectangulosxx.csv" donde xx está en el conjunto {00,01, 02, ..., 10} contienen familias de rectángulos de prueba.

Este ejercicio tiene 3 partes 

a) Escribir una función ModeloMISR que cree el modelo PE1 (o PL1) a partir de un archivo y que fije opciones estándar a fijar luego.

b) Escribir una función ResolverMISR(modeloR) que optimice el modelo ``modeloR`` y, en caso de encontrar óptimo, dibuje la salida
   
c) Ejecutar la función ResolverMISR en tres instancias de prueba.


### 1.a)

Escriba una función ModeloMISR que siga la siguiente sintaxis de entrada y salida:

```julia
function ModeloMISR(familia, integral=1)
return modeloR
```
donde la salida es

* `modeloR` es un objeto de tipo Model (creado por JuMP) basado en la formulación (PE).

y la entrada es

* `familia`:  Matriz que tiene los rectángulos de la familia

El parámetro "integral" puede ser 0 o 1 y es opcional. Si es 1 debe crear el modelo (PE), si es 0 debe crear el modelo (PL)


Su función debe fijar los siguientes parámetros en el modelo 

 "OutputFlag" 1  <- activa la salida del solver
 
 "TimeLimit" 30  <- limite de tiempo
 
 "Presolve" 0   <- desactiva el presolver
 
 "Threads" 1     <- desactiva el paralelismo (1 solo thread se permite)
 
 "Cuts" 0        <- desactiva la generación de cortes.
 
 "Heuristics" 0  <- desactiva las heuristicas de búsqueda.


In [ ]:
function ModeloMISR(familia, integral=1)
    # Paso 1, crear modelo. Aqui se usa el parámetro cotatiempo
    modeloR = Model(optimizer_with_attributes(
                () -> Gurobi.Optimizer(GUROBI_ENV)))
    
    # Paso 2 cree las variables, objetivo, y restricciones  del modelo
    
    # sugerencia: use if-else para determinar si cada x es Binaria, o si 0<=x<=1
    @variable(..)

    @objective(..)
    
    #sugerencia use restricciones sin nombre
    # for ...
    #     if(condicion)
    #          @constraint(modeloR, ...)
    #     end
    # end
    @constraint(..) 
        
    
    # Paso 4, active los 5 parámetros solicitados 
    Modifica(...)
        
    return modeloR
end

1.b) 
Escriba una función ResolverMISR que siga la siguiente sintaxis de entrada y salida:

```julia
function ResolverMISR(modeloR)
    # si no alcanza a terminar (por limite de tiempo)
    # debe arrojar un @error("Limite de tiempo alcanzado")
    #
    # antes de retornar, debe dibujar el conjunto S
    return valor_objetivo, S
```

Aquí `valor_objetivo` es el valor objetivo del problema, y `S` es la submatriz de R con los rectángulos ** en el soporte de la solución ** es decir, los rectángulos $r$ con $x_r>=eps$ para una tolerancia $eps=0.001$.

Para escribir la función puede usar la siguiente plantilla.

In [ ]:
function ResolverMISR(modeloR)
    
    # optimizar, extraer objetivos y valores
    
    optimize!(modeloR)
    if termination_status(modeloR) == MOI.TIME_LIMIT
        @error("Límite de tiempo alcanzado")
    end
    
    valor_objetivo = ..
    eps = 0.001
    S = ..  #matriz asociada a los coeficientes de x con valor >=eps
    
    # dibujar S
    
    # entregar
    return ..
    
end

### 1.c) 
Ejecute la función ResolverMISR en las instancia de pruebas "rectangulosxx.csv" con xx=01, 02, 03.
Haga esto para cada una dos veces, de manera integral y fraccional.


Verifique si hay alguna instancia en la que la solución integral y la solución fraccional son idénticas.

**PREGUNTA**: ¿Por qué cree que esto ocurre? Hint: Visualice las instancias

In [ ]:
ResolverMISR(ModeloMISR(LeeArchivo("rectangulos01.csv"),1))

In [ ]:
ResolverMISR(ModeloMISR(LeeArchivo("rectangulos01.csv"),0))

# D. Paralelismo y prioridad de branching

Las instancias "rectangulosxx.csv" para xx>=04 son bastante más complejas que las 3 primeras y para abordarlas manipularemos los parámetros del solver. 

### D1 Paralelismo.

El parámetro **Threads** (hilos) de Gurobi recibe un número natural $n$. Si $n$ es positivo, entonces Gurobi intentará usar hasta $n$ cores (si es que hay disponibles) para recorrer el árbol de branch and bound. Si $n=0$, Gurobi elige automáticamente el número de hilos a usar (usualmente es el máximo disponible).

En general, activar el paralelismo mejora el rendimiento de Gurobi. Son pocos los casos donde conviene usar solo un hilo. Por ejemplo, podría servir si la memoria es escasa o si hay otros procesos importantes corriendo en el mismo computador. También sirve en instancias en las que cerrar el gap de integralidad es complicado.

Puede cambiar el número de threads asignado a un modelo usando la función ``Modifica`` que se escribió al principio del laboratorio.

```julia
Modifica(mimodelo, ["Threads" 0]) # Activa paralelismo
Modifica(mimodelo, ["Threads" 1]) # Desactiva paralelismo: 1 solo thread
```

### D2 Prioridad de branching.

Recordemos que cada vez que el solver encuentra un óptimo (fraccional) de un nodo del árbol de branch and bound, debe decidir en que variable fraccional subdividir (hacer "branching"). Usualmente el solver elige adecuadamente de acuerdo a heurísticas internas. Sin embargo, también permite al usuario decidir que variables desea subdividir primero. Para esto, el usuario debe asignar un valor entero, llamado prioridad de branching a cada variable. Cada vez que el solver deba escoger una variable para subdividir, eligirá la variable fraccional con mayor prioridad asignada. 

En ocasiones el usuario (modelador) puede determinar que variables tienen más influencia en la integralidad de la solución. Por ejemplo, si existe una variable $y$ tal que al fijar $y$ a ser entera automáticamente todas las otras se hacen enteras, puede convenir darle mayor prioridad a dicha variable. 

En julia, para cambiar la prioridad de la variable ``x[i]`` del modelo ``m`` al valor ``nuevaprioridad`` puede ejecutar el siguiente comando

```julia
MOI.set(m, Gurobi.VariableAttribute("BranchPriority"), m[:x][i], nuevaprioridad)
```

En este ejercicio revisaremos el efecto de cambiar la prioridad de branching, dandole mayor prioridad a los rectángulos con mayor área discreta (usaremos nuevaprioridad = areadiscreta del rectángulo correspondiente)

## Ejercicio 2:

### 2.1
Diseñe una función ``test`` que siga la siguiente sintaxis de entrada y salida:

```julia
function test(nombrearchivo)
    # Entrega una tabla df (DataFrame) con 3 atributos 
    # "threads"(0/1), "prioridades" (0/1), "tiempo" (Float: tiempo de ejecución)
    return df
```

Su tabla se podría ver como sigue (tabla con valores ficticios)

In [ ]:
df = DataFrame(threads = Int8[], prioridades = Int8[], tiempo = Float64[])
push!(df, [0, 0, 0.12])
push!(df, [1, 0, 0.50])
push!(df, [0, 1, 1.99])
push!(df, [1, 1, 5.99])
show(df)

Por ejemplo, la tercera fila de la tabla anterior, se interpreta como que la instancia se resolvió en 1.99 segundos al activar la opción "threads" 0, y al usar como prioridades las areas discretas.

Puede completar la plantilla de la siguiente celda para entregar su función.

** Indicación: ** Use el comando ``t=@elapsed optimize!(m)`` para guardar en ``t``  el tiempo que demora en optimizar el modelo ``m``.


In [ ]:
function test(nombrearchivo)
    # Recupera familia R
    R = ..
    # Crear dataframe vacio
    df = ..
    for threads in 0:1
        for prioridades in 0:1
            # Crear modelo con opciones estándar
            m = ModeloMISR(R)
            # Fijar opciones threads 
            ..
            # Cambiar prioridades si es necesario
            if (prioridades == 1)
                ..
            end
            
            # Optimizar y medir tiempo
            t= ..
            
            # Agregar datos pedidos al dataframe
            push!(df, ..)
            
        end
    end
    
    return df
end

### 2.2 
Pruebe su función test en las 5 primeras instancias de la lista. Cuando termine responda la pregunta al final.

In [ ]:
test("rectangulos01.csv")

¿Cuál fue la mejor combinación de parámetros encontrados?

# E. Uso de Cortes

## Ejercicio 3: Activación y desactivación de cortes automáticos.

Los solvers como Gurobi no usan Branch and Bound puro, sino que usan una variante conocida como Branch and Cut.
En esta variante cada vez que un nodo del árbol de B&B se crea, el solver puede añadir restricciones (cortes)
generadas de manera automática. En las últimas clases de este curso veremos algunos de estos en detalle.
En este laboratorio, la idea será principalmente entender cual es el efecto de activar o desactivar ciertos cortes 
en el tiempo de resolución.

El parámetro CUTS de gurobi puede tomar valores -1 = automático; 0 = desactivado; 1-2-3 = nivel de agresividad en la generación de cortes de cualquier tipo a no ser que algún tipo específico de corte se active.
Gurobi genera cortes específicos, como por ejemplo CliqueCuts, ZeroHalfCuts y RLTCuts. Cada uno de ellos tiene un parámetro asociado que toma el valor -1 = automático; 0 = desactivado; 1-2-3 (o 1-2 en algunos casos) para el nivel de agresividad.

En este ejercicio desactivaremos los cortes de manera global y probaremos la eficiencia al activar (fijar en 1) la generación de CliqueCuts, ZeroHalfCuts y RLTCuts. En la última parte del laboratorio estudiaremos los CliqueCuts en detalle.

### 3.1
Diseñe una función ``testcuts`` que siga la siguiente sintaxis de entrada y salida:

```julia
function testcuts(nombrearchivo)
    # Entrega una tabla df (DataFrame) con 4 atributos 
    # "CliqueCuts"(0/1), "ZeroHalfCuts" (0/1), "RLTCuts" (0/1), "tiempo" (Float: tiempo de ejecución)
    return df
```
de manera similar al ejercicio anterior. Puede seguir la siguiente plantilla






In [ ]:
function testcuts(nombrearchivo)
    # Recupera familia R
    R = ..
    # Crear dataframe vacio
    df = ..
    for ..
        for ..
            for ..
                # Crear modelo con opciones estándar
                m = ModeloMISR(R)
                
                # Active Threads = 0  (de lo contrario podrìa ser muy lento su ejecución)
                Modifica(m, ["Threads" 0])
            
                # Fijar resto opciones 
                ..
                # Optimizar y medir tiempo (nota: si llega a tiempo limite, t será aproximadamente 30s)
                t= ..
            
                # Agregar datos pedidos al dataframe
                push!(df, ..)
            end
        end
    end
    
    return df
end

### 3.2
Pruebe su función testcuts en las instancias 6 a 10 de la lista. 
Observe que luego de optimizar el log de gurobi indica el número de cortes de cada tipo usado en la ejecución.

Cuando termine responda la pregunta al final.


In [ ]:
testcuts("rectangulos06.csv")

¿Cuál fue la mejor combinación de parámetros encontrados?

## Ejercicio 4: CliqueCuts de forma manual.

Si en un programa binario con variables $\{x_i\}_{i\in [n]}$, dos variables $x_i$ y $x_j$ son tales que ambas no pueden ser 1 simultáneamente, decimos que $\{i, j\}$ es un **par conflictivo**. Un clique es un conjunto $C \subseteq [n]$ tales que para todo $i, j \in C$, $\{i, j\}$ es conflictivo. Un **clique cut** es cualquier desigualdad de la forma $$\sum_{i\in C}x_i \leq 1$$ donde C es un clique maximal.

Es fácil ver que los clique cuts son desigualdades válidas para todo programa binario (de hecho, la desigualdad es válida incluso si el clique no es maximal), pero que un programa binario podría tener un número exponencial de clique cuts. 

GUROBI  es capaz de generar clique cuts de manera bastante razonable pero es más efectivo (en general) introducir estos cortes de manera explícita si es posible hacerlo.

Para el problema de conjunto independiente de rectángulos, es relativamente sencillo notar que todo clique máximo
corresponde a un conjunto de rectángulos que contienen a un mismo punto. Más aún, si $C$ es un clique entonces el punto
con coordenadas más pequeñas contenido en todos los rectángulos del clique $C$ es 
$(p_C,q_C)=(\max_{i\in C}a_i, \max_{i\in C}b_i)$,

Notamos que $(p_C,q_C)$ es un punto de la grilla $A\times B$ donde $A=\{a_i\colon i\in [n]\}$ y $B=\{b_j\colon j\in [n]\}$. 
Gracias a la discusión anterior todos los clique cuts son de la forma
$$[CQ(p,q)]:\quad \sum_{i\in [n]\colon \text{rect}(a_i,b_i,c_i,d_i) \ni (p,q)} x_i \leq 1$$

Más aún, los clique cuts implican la desigualdad no trivial del modelo (PE), por lo cual podemos crear un modelo (PE2) más fuerte que (PE) como sigue:

$$(PE2)\quad \begin{align*}
  \max &\sum_{i=1} x_i\\
  \sum_{i\in [n]\colon \text{rect}(a_i,b_i,c_i,d_i) \ni (p,q)} x_i &\leq 1\quad  \forall (p,q) \in A\times B\\
  x_i &\in \{0,1\}, \quad \forall i\in [n]
  \end{align*}
  $$
  

### 4.1

Cree una función
```julia
function ModeloMISR2(familia)
return modeloR
```
donde la salida es

* `modeloR` es un objeto de tipo Model (creado por JuMP) basado en la formulación (PE).

y la entrada es

* `familia`:  Matriz que tiene los rectángulos de la familia

Su función debe fijar los siguientes parámetros en el modelo 

 "OutputFlag" 1  <- activa la salida del solver
 
 "TimeLimit" 30  <- limite de tiempo
 
 "Presolver" 0   <- desactiva el presolver
 
 "Threads" 1     <- desactiva el paralelismo (1 solo thread se permite)
 
 "Cuts" 0        <- desactiva la generación de cortes.
 
 "Heuristics" 0  <- desactiva las heuristicas de búsqueda.

In [ ]:
function ModeloMISR2(familia)
..
    # Cuidados: Cuando agregue las restricciones al modelo asegúrese de poner solo una restricción por cada par en A x B
    # (podrian haber multiples puntos (a_i,b_j) iguales para distintos pares (i,j))
    # Le puede servir el comando ?unique
..
return modeloR

## 4.2  

Diseñe una función ``testcliquecuts`` que siga la siguiente sintaxis de entrada y salida:

```julia
function testcliquecuts(nombrearchivo)
    # Entrega una tabla df (DataFrame) con 2 atributos 
    # "modelo" (1/2), "tiempo" (Float: tiempo de ejecución)
    return df
```
de manera similar al ejercicio anterior. 

Aquí "modelo 1" significa resolver la instancia:
* usando ModeloMISR1
* activando el parámetro CliqueCuts=1 de gurobi (pero desactivando todos los otros cortes)

y "modelo 2" significa resolver la instancia:
* usando ModeloMISR2
* sin activar ningún corte ( CliqueCuts tampoco debe estar activado)



### 4.3
Pruebe su función testcliquecuts en las instancias 6 a 10 de la lista. 
Cuando termine responda la pregunta al final.


¿Cuál fue la mejor opción encontrada?